In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df.review[1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

#### Text Cleaning
    1. Sample 10000 rows
    2. Remove HTML tags
    3. Remove Special Charecters
    4. Converting every letter to lowercase
    5. Removing Stop Words
    6. Stemming

In [4]:
df = df.sample(10000)
df.shape

(10000, 2)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 3533 to 27643
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     10000 non-null  object
 1   sentiment  10000 non-null  object
dtypes: object(2)
memory usage: 234.4+ KB


In [6]:
df['sentiment'].replace({'positive': 1, 'negative' : 0}, inplace = True)
df.head()

/tmp/ipykernel_33/626607573.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['sentiment'].replace({'positive': 1, 'negative' : 0}, inplace = True)
/tmp/ipykernel_33/626607573.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'].replace({'positive': 1, 'negative' : 0}, inplace = 

,review,sentiment
3533,Claire Denis has demonstrated repeatedly that ...,1
17880,I've read approximately 10 reviews of this fil...,1
427,This is an OK adaptation of the breath taking ...,1
24611,When his in-laws are viciously murdered by a g...,0
11818,At first i didn't like they way the director w...,1


In [7]:
#removing html tags
import re
clean = re.compile('<.*?>')
re.sub(clean, '', df.iloc[2].review)

"This is an OK adaptation of the breath taking book of Dan Brown. I can't say it is novel or very good but they made a movie that you can enjoy. Given the excellent story, the result could have been better though. The movie is pretty long but at the end I was feeling like some things were missing. Sound effects and sound tracks were very good. Acting was well done but the character development phase was very weak. For people who didn't read the book, things may look happening too quickly. From my point of view, instead of trying to put as much as stuff from the book, they could have tried to do the important scenes more proper. What makes the book very good was all the puzzle like story combined with the excellent portrait of Vatican. You see neither of it in the movie. Too much rush and using the time not in a good way, these are main problems of the movie. So, it is worth watching but could have been done better."

In [8]:
def clean_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [9]:
df['review'] = df['review'].apply(clean_html)

In [10]:
df.iloc[1].review

'I\'ve read approximately 10 reviews of this film, but haven\'t taken any of them "to heart". Harrison Ford is being criticized for everything from his haircut to his earring and I really don\'t see what any of these things have to do with the film. As I recall, he had bad hair plus bad reviews in Blade Runner but it\'s still a favorite among us videophiles. It is slower than the majority of his previous films but not worthy of the trashings it has received.I would gladly pay $7.50 just to watch a bald Harrison Ford mow his lawn.'

In [11]:
# function for removing special charecters

def remove_special(text):
    x = ''

    for i in text:
        if i.isalnum():
            x = x + i
        else:
            x = x + ' '
    return x

In [12]:
remove_special(' th%e @ classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare. forget pretty pictures painted for mainstream audiences, f')

' th e   classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare  forget pretty pictures painted for mainstream audiences  f'

In [13]:
df.review = df.review.apply(remove_special)

In [14]:
df.iloc[1].review

'I ve read approximately 10 reviews of this film  but haven t taken any of them  to heart   Harrison Ford is being criticized for everything from his haircut to his earring and I really don t see what any of these things have to do with the film  As I recall  he had bad hair plus bad reviews in Blade Runner but it s still a favorite among us videophiles  It is slower than the majority of his previous films but not worthy of the trashings it has received I would gladly pay  7 50 just to watch a bald Harrison Ford mow his lawn '

In [15]:
#Converting to Lower case
df.review = df.review.apply(lambda x : x.lower())

In [16]:
df.iloc[1].review

'i ve read approximately 10 reviews of this film  but haven t taken any of them  to heart   harrison ford is being criticized for everything from his haircut to his earring and i really don t see what any of these things have to do with the film  as i recall  he had bad hair plus bad reviews in blade runner but it s still a favorite among us videophiles  it is slower than the majority of his previous films but not worthy of the trashings it has received i would gladly pay  7 50 just to watch a bald harrison ford mow his lawn '

In [17]:
#Removing Stop Words
import nltk
from nltk.corpus import stopwords

In [18]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [19]:
def remove_stopwords(text):
    x = []
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y = x[:]
    x.clear()
    return y

In [20]:
df.review = df.review.apply(remove_stopwords)

In [21]:
df

,review,sentiment
3533,"[claire, denis, demonstrated, repeatedly, film...",1
17880,"[read, approximately, 10, reviews, film, taken...",1
427,"[ok, adaptation, breath, taking, book, dan, br...",1
24611,"[laws, viciously, murdered, gang, thugs, young...",0
11818,"[first, like, way, director, constantly, switc...",1
...,...,...
19564,"[first, scanners, may, great, movie, least, or...",0
14112,"[absolute, grabber, movie, given, age, years, ...",1
45965,"[movie, like, many, others, remember, titans, ...",0
6519,"[sistas, da, hood, looking, revenge, bling, bl...",0


In [23]:
#Stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [25]:
y = []

def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z = y[:]
    y.clear()
    return z

In [26]:
stem_words(['I','loved','loving','it'])

['I', 'love', 'love', 'it']

In [28]:
df.review = df.review.apply(stem_words)

In [29]:
df

,review,sentiment
3533,"[thought, wast, preciou, 50, buck, go, watch, ...",1
17880,"[read, approxim, 10, review, film, taken, hear...",1
427,"[ok, adapt, breath, take, book, dan, brown, sa...",1
24611,"[law, vicious, murder, gang, thug, young, depu...",0
11818,"[first, like, way, director, constantli, switc...",1
...,...,...
19564,"[first, scanner, may, great, movi, least, orig...",0
14112,"[absolut, grabber, movi, given, age, year, ahe...",1
45965,"[movi, like, mani, other, rememb, titan, mirac...",0
6519,"[sista, da, hood, look, reveng, bling, bling, ...",0


In [30]:
df.review = df.review.apply(lambda x : " ".join(x))

In [31]:
df

,review,sentiment
3533,thought wast preciou 50 buck go watch movi ear...,1
17880,read approxim 10 review film taken heart harri...,1
427,ok adapt breath take book dan brown say novel ...,1
24611,law vicious murder gang thug young deputi orde...,0
11818,first like way director constantli switch past...,1
...,...,...
19564,first scanner may great movi least origin nove...,0
14112,absolut grabber movi given age year ahead time...,1
45965,movi like mani other rememb titan miracl follo...,0
6519,sista da hood look reveng bling bling except d...,0


In [55]:
X = df.iloc[:,0:1].values

In [56]:
X.shape

(10000, 1)

In [57]:
from sklearn.preprocessing import Binarizer

binarizer = Binarizer()

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2500)

In [75]:
X = cv.fit_transform(df['review']).toarray()

In [85]:
X = binarizer.fit_transform(X)

In [61]:
X.shape

(10000, 2500)

In [76]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [77]:
X[0].mean()

0.1068

In [63]:
y = df.iloc[:,-1].values

In [64]:
y.shape

(10000,)

In [86]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [79]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [88]:
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()

clf1.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)

y_pred1=clf1.predict(X_test)
y_pred2=clf2.predict(X_test)
y_pred3=clf3.predict(X_test)

In [89]:
from sklearn.metrics import accuracy_score
print("Gaussian",accuracy_score(y_test,y_pred1))
print("Multinomial",accuracy_score(y_test,y_pred2))
print("Bernaulli",accuracy_score(y_test,y_pred3))

Gaussian 0.7595
Multinomial 0.842
Bernaulli 0.8445


In [87]:
from sklearn.metrics import accuracy_score
cf3 = BernoulliNB()

cf3.fit(X_train, y_train)

y_pred = clf3.predict(X_test)

print("Bernoulli",accuracy_score(y_test,y_pred))

Bernoulli 0.858
